In [2]:
import csv

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [3]:
def read_add_click_data(n,offset = 0):
    X_dict,y = [],[]
    with open(r'E:\programming\dataset\click_throug_predictions\train\train.csv','r') as csvfile:
        reader = csv.DictReader(csvfile)
        for i in range(offset):
            next(reader)
        i = 0
        for row in reader:
            i+=1
            y.append(row['click'])
            del row['click'],row['id'],row['hour'],row['device_id'],row['device_ip']
            X_dict.append(dict(row))
            if i>=n:
                break
    return X_dict,y            


In [4]:
n = 10000
X_dict_train,y_train = read_add_click_data(n)
print(len(X_dict_train[0]))

X_dict_test,y_test = read_add_click_data(2000,n)

19


In [5]:
#one hot encoding
dic_vectorizer = DictVectorizer(sparse = False)
X_train = dic_vectorizer.fit_transform(X_dict_train)
X_test = dic_vectorizer.transform(X_dict_test)
len(X_train[0])

2820

In [1]:
#the dimension are increased from 19 to 2820

In [7]:
paramters = {'max_depth':[3,10,None]}
decision_tree = DecisionTreeClassifier(criterion = 'gini',min_samples_split=30)

grid_search = GridSearchCV(decision_tree,paramters,n_jobs = -1,cv = 3,scoring='roc_auc')

In [8]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=30,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1, param_grid={'max_depth': [3, 10, None]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring

In [9]:
grid_search.best_params_

{'max_depth': 10}

In [14]:
best_tree = grid_search.best_estimator_
prob = best_tree.predict_proba(X_test)[:,1]
print('The score is {}'.format(roc_auc_score(y_test,prob)))

The score is 0.6856094856117417
